# Podjetje, varianta 1

In [ ]:
from sqlalchemy import Column, DateTime, String, Integer, ForeignKey, func, select, create_engine
from sqlalchemy.orm import declarative_base, relationship, backref, sessionmaker
from datetime import datetime

In [ ]:
Base = declarative_base()


class Oddelek(Base):
    __tablename__ = 'oddelek'

    id = Column(Integer, primary_key=True)
    naziv = Column(String)

    def __repr__(self):
        return f"Oddelek[{self.id}, {self.naziv}, {self.seznamZaposlenih}]"


class Zaposleni(Base):
    __tablename__ = 'zaposleni'

    id = Column(Integer, primary_key=True)
    ime = Column(String)

    # uporabimo razred func za dostop do funkcij na bazi, npr. func.now()
    datum_zaposlitve = Column(DateTime, default=func.now())
    oddelek_id = Column(Integer, ForeignKey('oddelek.id'))

    # cascade='delete,all' bo povzročil brisanje vseh zaposlenih v oddelku
    oddelek = relationship(Oddelek,
                           backref=backref('seznamZaposlenih', uselist=True, cascade='delete,all'))

    def __repr__(self):
        return f"Zaposleni[{self.id}, {self.ime}, {self.datum_zaposlitve}, {self.oddelek_id}]"

In [ ]:
engine = create_engine('sqlite:///podjetje1.db', echo=True)
Base.metadata.bind = engine
Base.metadata.create_all(engine)

In [ ]:
DBSessionMaker = sessionmaker(bind=engine)
session = DBSessionMaker()

Ustvarimo oddelek.

In [ ]:
(oddelek1 := Oddelek(naziv="IT"))

Ustvarimo zaposlenega.

In [ ]:
(zaposleni1 := Zaposleni(ime="Janez", oddelek=oddelek1))

Dodajmo ustvarjena objekta v bazo.

In [ ]:
session.add(oddelek1)
session.add(zaposleni1)
session.commit()

Izpišimo trenutno vsebino baze.

In [ ]:
session.query(Zaposleni).all()

In [ ]:
session.query(Oddelek).all()

Izbrišimo `oddelek1`.

In [ ]:
session.delete(oddelek1)
session.commit()

Spet izpišimo trenutno vsebino baze.

In [ ]:
session.query(Zaposleni).all()

In [ ]:
session.query(Oddelek).all()

Ustvarimo naslednjega zaposlenega in izpišimo datum zaposlitve.

In [ ]:
zaposleni2 = Zaposleni(ime="Francka")
session.add(zaposleni2)
print(zaposleni2.datum_zaposlitve)

Privzeta vrednost se nastavi šele, ko kličemo `commit`.

In [ ]:
session.commit()
print(zaposleni2.datum_zaposlitve)

Za privzeto vrednost smo uporabili `func.now()`. Izpišimo njeno vrednost pred in po interakciji z bazo.

In [ ]:
func.now()

In [ ]:
rs = session.execute(select(func.now()))
rs.fetchone()

Pobrišimo vse oddelke in uporabnike v njih.

In [ ]:
for oddelek in session.query(Oddelek).all():
    session.delete(oddelek)
session.commit()

Še enkrat preglejmo zaposlene.

In [ ]:
session.query(Zaposleni).all()

Ustvarimo dva oddelka in tri zaposlene, ki jih vstavimo v oddelke.

In [ ]:
IT = Oddelek(naziv="IT")
finance = Oddelek(naziv="Finance")
janez = Zaposleni(ime="Janez", oddelek=IT)
metka = Zaposleni(ime="Metka", oddelek=finance)
session.add(IT)
session.add(finance)
session.add(janez)
session.add(metka)
session.commit()

In [ ]:
katka = Zaposleni(ime="Katka", oddelek=finance)
session.add(katka)
session.commit()

Preštejmo zaposlene.

In [ ]:
session.query(Zaposleni).count()

Izpišimo ime enega zaposlenega, ki se začne na črko K.

In [ ]:
session.query(Zaposleni).filter(Zaposleni.ime.startswith("K")).one().ime

Izpišimo ime enega zaposlenega, ki je hkrati v oddelku Finance in se njegovo ime začne na črko K.

In [ ]:
session.query(Zaposleni).join(Zaposleni.oddelek).filter(Zaposleni.ime.startswith('K'), Oddelek.naziv == 'Finance').all()

Izpišimo število zaposlenih, ki so bili zaposleni v preteklosti.

In [ ]:
session.query(Zaposleni).filter(Zaposleni.datum_zaposlitve < func.now()).count()